In [1]:
import pandas as pd
import altair as alt

cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

## 6.6. Detalhes sob Demanda

Assim que encontramos pontos de interesse dentro da visualização, geralmente queremos saber mais sobre eles. _Detalhe sob demanda_ se refere consultar interativamente buscando mais informações sobre valores selecionados. `Tooltips` são um recurso útil para fornecer detalhes sob demanda. Porém, `tooltips` tipicamente mostram apenas informações para um ponto do dado por vez. Como podemos mostrar mais?

O gráfico de dispersão das classificações dos filmes inclui valores discrepantes (outliers) potencialmente interessantes onde o IMDB e o Rotten Tomatoes discordam. Vamos criar um plot que nos permite selecionar pontos interativamente e mostrar seus rótulos. Para acionar a consulta de filtro através da interação de pairar ou da interação de clicar, usaremos o [operador de composição do Altair](https://altair-viz.github.io/user_guide/interactions.html#composing-multiple-selections) `|` ("ou").

_Passe o mouse sobre pontos no gráfico de dispersão abaixo para ver um rótulo de destaque e título. Use Shift+Click nos pontos para fazer anotações persistentes e ver vários rótulos ao mesmo tempo. Quais filmes são amados pela crítica do Rotten Tomatoes, mas não pelo público geral do IMDB (e vice versa)? Veja se você consegue achar possíveis erros, onde dois filmes diferentes com o mesmo nome foram acidentalmente combinados!_

In [ ]:
hover = alt.selection_single(
    on='mouseover',  # selecione ao passar o mouse
    nearest=True,    # selecione o ponto mais próximo do cursor do mouse
    empty='none'     # a seleção vazia não deve corresponder a nada
)

click = alt.selection_multi(
    empty='none' # seleção vazia não corresponde a nenhum ponto
)

# codificações de gráficos de dispersão compartilhadas por todas as marcas
plot = alt.Chart().mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q'
)
  
# base compartilhada para novas camadas
base = plot.transform_filter(
    hover | click # filtrar para pontos em qualquer seleção
)

# pontos do gráfico de dispersão de camadas, anotações de halo e rótulos de título
alt.layer(
    plot.add_selection(hover).add_selection(click),
    base.mark_point(size=100, stroke='firebrick', strokeWidth=1),
    base.mark_text(dx=4, dy=-8, align='right', stroke='white', strokeWidth=2).encode(text='Title:N'),
    base.mark_text(dx=4, dy=-8, align='right').encode(text='Title:N'),
    data=movies
).properties(
    width=600,
    height=450
)

O exemplo acima adiciona três novas camadas para o gráfico de dispersão: uma anotação circular, texto branco para fornecer um fundo legível e texto em preto mostrando o nome do filme. Ademais, esse exemplo usa duas seleções em conjunto:

1. Uma seleção única (`hover`) que inclui `nearest=True` para automaticamente selecionar o ponto do dado que está mais próximo do cursor.
2. Uma seleção multi (`click`) para criar seleções persistentes via Shift+Click.

Ambas as seleções incluem o `empty='none'` para indicar que nenhum ponto deve ser incluso se a seleção está vazia. Essas seleções são então combinadas num único predicado de filtro &mdash; o _ou_ lógico de `hover` e `click` &mdash; para incluir pontos que estão em _qualquer_ uma das duas seleções. Usamos esse predicado para filtar as novas camadas para mostrar anotações e rótulos apenas para pontos selecionados.

Usando seleções e camadas, podemos fazer vários designs diferentes sob demanda! Por exemplo, aqui está uma série temporal de preços de ações de tecnologia em escala logarítmica, anotadas com uma diretriz e rótulos para a data mais próxima ao cursor:

In [ ]:
# selecione um ponto para o qual fornecer detalhes sob demanda
label = alt.selection_single(
    encodings=['x'], # limitar a seleção ao valor do eixo x
    on='mouseover',  # selecione em eventos de mouseover
    nearest=True,    # selecione o ponto de dados mais próximo do cursor
    empty='none'     # seleção vazia não inclui pontos de dados
)

# definir nosso gráfico de linha base de preços de ações
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, #gráfico de linha básica
    
    # adicione uma marca de regra para servir como linha guia
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),
    
    # adicionar marcas circulares para pontos de tempo selecionados, ocultar pontos não selecionados
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # adicione texto com traços brancos para fornecer um fundo legível para rótulos
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # adicione rótulos de texto para preços de ações
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),
    
    data=stocks
).properties(
    width=700,
    height=400
)

_Colocando o que aprendemos até agora em prática: você consegue mudar o gráfico de dispersão acima (aquele com a consulta dinâmica nos anos) para incluir uma marca de regra (`mark_rule`) que mostra a média das classificações IMDB (ou Rotten Tomatoes) para os dados contidos na seleção `interval` dos anos?_